In [1]:
import numpy as np
from operator import itemgetter 
from gurobipy import *

products = {"cr1", "cr2", "ln", "mn", "hn", "lo", "ho", "r", "rg", "cg", "co", "pmf", "rmf", "jf", "fo", "lbo"}
distill = {"cr1", "cr2"}
reform = {"ln", "mn", "hn"}
crack = {"lo", "ho"}
profit = {"pmf":7, "rmf":6, "jf": 4, "fo": 3.5, "lbo":1.5}
distill_cap = 45000
reform_cap = 10000
crack_cap = 8000
buy_limit = {"cr1": 20000, "cr2": 30000}
lbo_min = 500
lbo_max = 1000
yields = {
    ("cr1", "ln"): 0.10,
    ("cr2", "ln"): 0.15,
    ("cr1", "mn"): 0.20,
    ("cr2", "mn"): 0.25,
    ("cr1", "hn"): 0.20,
    ("cr2", "hn"): 0.18,
    ("cr1", "lo"): 0.12,
    ("cr2", "lo"): 0.08,
    ("cr1", "ho"): 0.20,
    ("cr2", "ho"): 0.19,
    ("cr1", "r"): 0.13,
    ("cr2", "r"): 0.12,
    ("ln", "rg"): 0.60,
    ("mn", "rg"): 0.52,
    ("hn", "rg"): 0.45,
    ("lo", "co"): 0.68,
    ("ho", "co"): 0.75,
    ("lo", "cg"): 0.28,
    ("ho", "cg"): 0.20,
    ("r", "lbo"): 0.50,
    ("ln", "pmf"): 1.00,
    ("mn", "pmf"): 1.00,
    ("hn", "pmf"): 1.00,
    ("rg", "pmf"): 1.00,
    ("cg", "pmf"): 1.00,
    ("ln", "rmf"): 1.00,
    ("mn", "rmf"): 1.00,
    ("hn", "rmf"): 1.00,
    ("rg", "rmf"): 1.00,
    ("cg", "rmf"): 1.00,
    ("lo", "jf"): 1.00,
    ("ho", "jf"): 1.00,
    ("co", "jf"): 1.00,
    ("r", "jf"): 1.00,
    ("lo", "fo"): 1.00,
    ("ho", "fo"): 1.00,
    ("co", "fo"): 1.00,
    ("r", "fo"): 1.00
}
used_for = {
    "ln": {"rg", "pmf", "rmf"},
    "mn": {"rg", "pmf", "rmf"},
    "hn": {"rg", "pmf", "rmf"},
    "lo": {"cg", "co", "jf", "fo"},
    "ho": {"cg", "co", "jf", "fo"},
    "co": {"jf", "fo"},
    "r": {"lbo", "jf", "fo"},
    "cg": {"pmf", "rmf"},
    "rg": {"pmf", "rmf"},   
} 
fo_ratios = {"lo":5/9, "ho":1/6, "co":2/9, "r":1/18}
vapor_pressure = {"lo": 1, "ho":0.6, "co":1.5, "r":0.05}
octane_rating = {"ln":90, "mn": 80, "hn":70, "rg":115, "cg":105}
octane_tol = {"pmf":94, "rmf": 84}

In [2]:
refinery = Model()
get = refinery.addVars(yields.keys(), name="Get")
produce = refinery.addVars(products, name="Produce")

In [3]:
refinery.addConstrs((produce[crude] <= buy_limit[crude] for crude in distill), name="Purchase_limit")

{'cr2': <gurobi.Constr *Awaiting Model Update*>,
 'cr1': <gurobi.Constr *Awaiting Model Update*>}

In [4]:
refinery.addConstr(quicksum(produce[crude] for crude in distill), GRB.LESS_EQUAL, distill_cap, name="Distill_capacity")

<gurobi.Constr *Awaiting Model Update*>

In [5]:
refinery.addConstr(get.sum("*", "rg"), GRB.LESS_EQUAL, reform_cap, name="Reform_capacity")

<gurobi.Constr *Awaiting Model Update*>

In [6]:
refinery.addConstr(get.sum("*", "cg") + get.sum("*", "co"), GRB.LESS_EQUAL, crack_cap, name="Crack_capacity")

<gurobi.Constr *Awaiting Model Update*>

In [7]:
refinery.addConstrs((get.prod(yields, "*", out) == produce[out] for out in set(map(itemgetter(1), yields.keys()))), name="Yield")

{'jf': <gurobi.Constr *Awaiting Model Update*>,
 'ln': <gurobi.Constr *Awaiting Model Update*>,
 'co': <gurobi.Constr *Awaiting Model Update*>,
 'rmf': <gurobi.Constr *Awaiting Model Update*>,
 'rg': <gurobi.Constr *Awaiting Model Update*>,
 'lbo': <gurobi.Constr *Awaiting Model Update*>,
 'r': <gurobi.Constr *Awaiting Model Update*>,
 'hn': <gurobi.Constr *Awaiting Model Update*>,
 'cg': <gurobi.Constr *Awaiting Model Update*>,
 'pmf': <gurobi.Constr *Awaiting Model Update*>,
 'fo': <gurobi.Constr *Awaiting Model Update*>,
 'lo': <gurobi.Constr *Awaiting Model Update*>,
 'mn': <gurobi.Constr *Awaiting Model Update*>,
 'ho': <gurobi.Constr *Awaiting Model Update*>}

In [8]:
refinery.addConstrs((quicksum(get[iput, oput] for oput in used_for[iput]) == produce[iput] for iput in used_for.keys()), name="Mass_conservation")

{'ln': <gurobi.Constr *Awaiting Model Update*>,
 'mn': <gurobi.Constr *Awaiting Model Update*>,
 'hn': <gurobi.Constr *Awaiting Model Update*>,
 'lo': <gurobi.Constr *Awaiting Model Update*>,
 'ho': <gurobi.Constr *Awaiting Model Update*>,
 'co': <gurobi.Constr *Awaiting Model Update*>,
 'r': <gurobi.Constr *Awaiting Model Update*>,
 'cg': <gurobi.Constr *Awaiting Model Update*>,
 'rg': <gurobi.Constr *Awaiting Model Update*>}

## We need additional constraints to keep proportions right

In [9]:
refinery.addConstrs((get[iput, "fo"] == fo_ratios[iput]*produce["fo"] for iput in fo_ratios.keys()),name="Fueloil_ratios")
produce["lbo"].LB = lbo_min
produce["lbo"].UB = lbo_max

In [10]:
refinery.addConstr(produce["pmf"], GRB.GREATER_EQUAL, 0.40*produce["rmf"], name="Premium2Regular")

<gurobi.Constr *Awaiting Model Update*>

In [11]:
refinery.addConstrs((get.prod(octane_rating, "*", gas) >= octane_tol[gas]*produce[gas] for gas in {"pmf", "rmf"}), name="Octane_rating")

{'pmf': <gurobi.Constr *Awaiting Model Update*>,
 'rmf': <gurobi.Constr *Awaiting Model Update*>}

In [12]:
refinery.addConstr(get.prod(vapor_pressure, "*", "jf"), GRB.LESS_EQUAL, produce["jf"], name="Vapor_pressure")

<gurobi.Constr *Awaiting Model Update*>

In [13]:
refinery.setObjective(quicksum(profit[prod]*produce[prod] for prod in profit.keys()), GRB.MAXIMIZE)

In [14]:
refinery.optimize()

Optimize a model with 36 rows, 54 columns and 111 nonzeros
Coefficient statistics:
  Matrix range     [6e-02, 9e+01]
  Objective range  [2e+00, 7e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [8e+03, 4e+04]
Presolve removed 17 rows and 30 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible or unbounded model


In [15]:
refinery.write("file.lp")